# Load data

In [1]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [3]:
# Transformations
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load train set
train_set = dsets.CIFAR10('../', train=True, download=True, transform=transform_train)

# Load test set (using as validation)
val_set = dsets.CIFAR10('../', train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


# Import Resnet Models
----

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os.path
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from copy import deepcopy
from resnet import ResNet18

In [5]:
# Select device
# device = torch.device('cuda')
device = torch.device('cpu')

## Import Resnet model

In [6]:
# Check for model
if os.path.isfile('cifar resnet.pt'):
    # Load saved model
    print('Loading saved model')
    model_resnet = torch.load('cifar resnet.pt').to(device)
else:
    print('Model not available')

Loading saved model


## Import surrogate

In [7]:
from fastshap import ImageSurrogate
from fastshap.utils import MaskLayer2d, KLDivLoss, DatasetInputOnly

In [8]:
# Check for model
if os.path.isfile('cifar surrogate.pt'):
    print('Loading saved surrogate model')
    surr = torch.load('cifar surrogate.pt').to(device)
    surrogate = ImageSurrogate(surr, width=32, height=32, superpixel_size=2)

else:
    print('Model not available')

Loading saved surrogate model


## Import Explainer

In [9]:
from unet import UNet
from fastshap import FastSHAP

In [10]:
# Check for model
if os.path.isfile('cifar explainer.pt'):
    print('Loading saved explainer model')
    explainer = torch.load('cifar explainer.pt').to(device)
    fastshap = FastSHAP(explainer, surrogate, link=nn.LogSoftmax(dim=1))

else:
    print('Model not available')

Loading saved explainer model


# Import Vit Models
-----

## Import Vit Model

In [29]:
from vit import ViT
model_path = 'cifar_c10_vit.pt'

In [30]:
if os.path.isfile(model_path):
    # load saved model
    print("Loading saved model")
    model_vit = torch.load(model_path).to(device)
else:
    print('Model not available')

Loading saved model


In [31]:
torch.save(model.cpu(), model_path)

## Import surrogates

In [15]:
from fastshap import ImageSurrogate
from fastshap.utils import MaskLayer2d, DatasetInputOnly, KLDivLoss

In [16]:
surr_model_path = 'cifar_c10_vit_surrogate_lb20.pt'

In [17]:
# Check for model
if os.path.isfile(surr_model_path):
    print('Loading saved surrogate model')
    surr = torch.load(surr_model_path).to(device)
    surrogate = ImageSurrogate(surr, width=32, height=32, superpixel_size=2)
else:
    print('Model not available')

Loading saved surrogate model


## Import explainer

In [18]:
from unet import UNet
from fastshap import FastSHAP

In [19]:
explainer_path = 'cifar_explainer_unet_lb20.pt'

In [20]:
# Check for model
if os.path.isfile(explainer_path):
    print('Loading saved explainer model')
    explainer = torch.load(explainer_path).to(device)
    fastshap = FastSHAP(explainer, surrogate, link=nn.LogSoftmax(dim=1))

else:
    print('Model not available')

Loading saved explainer model


# Initialize dataset

In [210]:
dset = val_set
dset_size = 300
# dset_size = 10
targets = np.array(dset.targets)
num_classes = targets.max() + 1
classes = ['Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

# Plot individual images and values

In [211]:
import matplotlib.pyplot as plt

In [212]:
def plot_img(x):
    mean = np.array([0.4914, 0.4822, 0.4465])[:, np.newaxis, np.newaxis]
    std = np.array([0.2023, 0.1994, 0.2010])[:, np.newaxis, np.newaxis]
    im = x.numpy() * std + mean    
    im = im.transpose(1, 2, 0).astype(float)
    im = np.clip(im, a_min=0, a_max=1)
    plt.imshow(im, vmin=0, vmax=1)
    plt.show()

In [213]:
def plot_values(value):
#     plt.imshow(value, cmap='seismic',vmin=-m, vmax=m )
    plt.imshow(value, cmap='seismic')

# Collect performance results Using Resnet

## Create dataframe to store results

In [214]:
import pandas as pd
import numpy as np

In [215]:
columns =  ['y_label','y_A','y_prob_correct_A', 'y_prob_A', 'A','y_B','y_prob_correct_B', 'y_prob_B', 'B']
dset_idxs = np.array(range(dset_size))
results = pd.DataFrame(columns = columns, index = dset_idxs)

In [216]:
results.head()

,y_label,y_A,y_prob_correct_A,y_prob_A,A,y_B,y_prob_correct_B,y_prob_B,B
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Collect data 

In [217]:
model_A = model_resnet
model_B = model_vit

In [218]:
def get_pred(dset, dset_size, idx, model, model_name, x, y):
    y_pred = model(x).softmax(dim=1).detach()
    y_pred_class = np.argmax(y_pred)

    # store values

    results.iloc[idx]['y_' + model_name] = y_pred_class.item()
    results.iloc[idx]['y_prob_correct_' + model_name] = y_pred[0][y].item()
    results.iloc[idx]['y_prob_' + model_name] = y_pred[0][y_pred_class.item()].item()

    if (y == y_pred_class): 
        results.iloc[idx][model_name] = 1
    else:
        results.iloc[idx][model_name] = 0

In [219]:
def collect_data(dset, dset_size, results, model_A, model_B):
    for idx in range(dset_size):
        x, y = zip(*[dset[idx]])
        x = torch.stack(x)
        y = y[0]
        
        results.iloc[idx]['y_label'] = y
        get_pred(dset, dset_size, idx, model_A, 'A', x, y)
        get_pred(dset, dset_size, idx, model_B, 'B', x, y)

In [220]:
def check_results(idx,  results):
    x, y = zip(*[dset[idx]])
    x = x[0]
    y = y[0]
    print(f'Label = {classes[y]}, {y}')
    y_predicted = results.iloc[idx]['y_resnet_class'] 
    print(f'Predicted = {classes[y_predicted]} , {y_predicted}' )
    plot_img(x)

In [221]:
collect_data(dset, dset_size, results, model_A, model_B)

In [222]:
results

,y_label,y_A,y_prob_correct_A,y_prob_A,A,y_B,y_prob_correct_B,y_prob_B,B
0,3,3,0.366576,0.366576,1,3,1.0,1.0,1
1,8,8,0.307475,0.307475,1,8,0.999998,0.999998,1
2,8,1,0.180506,0.241541,0,8,0.999936,0.999936,1
3,0,1,0.019308,0.37162,0,0,0.846047,0.846047,1
4,6,1,0.06216,0.321299,0,6,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...
295,4,1,0.102048,0.165709,0,4,0.9983,0.9983,1
296,6,1,0.134055,0.25714,0,6,1.0,1.0,1
297,0,1,0.028894,0.570569,0,0,0.99972,0.99972,1
298,0,3,0.097698,0.267243,0,0,0.998641,0.998641,1


# Classify samples into sets


In [227]:
A_wrong_B_wrong = results[(results['A'] == 0) & (results['B'] == 0)] 

In [228]:
A_wrong_B_correct = results[(results['A'] == 0) & (results['B'] == 1)] 

In [229]:
A_correct_B_wrong = results[(results['A'] == 1) & (results['B'] == 0)] 

In [230]:
A_correct_B_correct = results[(results['A'] == 1) & (results['B'] == 1)] 

In [231]:
A_wrong_B_wrong

,y_label,y_A,y_prob_correct_A,y_prob_A,A,y_B,y_prob_correct_B,y_prob_B,B
35,2,1,0.02934,0.325694,0,1,0.0,0.93951,0
47,9,1,0.189299,0.243881,0,8,0.038373,0.959992,0
53,3,9,0.105674,0.363836,0,6,0.000521,0.99941,0
57,7,3,0.043962,0.329053,0,5,0.0,0.99999,0
58,4,5,0.098013,0.228652,0,3,0.003957,0.98514,0
59,6,1,0.031643,0.327265,0,7,0.23121,0.679919,0
61,3,5,0.116148,0.219245,0,5,0.268519,0.731466,0
85,5,3,0.190139,0.225199,0,7,0.142349,0.801315,0
87,7,1,0.038673,0.414504,0,0,0.000032,0.995287,0
118,2,9,0.017582,0.376003,0,3,0.00156,0.993404,0


In [232]:
A_wrong_B_correct

,y_label,y_A,y_prob_correct_A,y_prob_A,A,y_B,y_prob_correct_B,y_prob_B,B
2,8,1,0.180506,0.241541,0,8,0.999936,0.999936,1
3,0,1,0.019308,0.37162,0,0,0.846047,0.846047,1
4,6,1,0.06216,0.321299,0,6,1.0,1.0,1
5,6,1,0.041133,0.551747,0,6,0.999997,0.999997,1
7,6,5,0.060609,0.264785,0,6,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...
295,4,1,0.102048,0.165709,0,4,0.9983,0.9983,1
296,6,1,0.134055,0.25714,0,6,1.0,1.0,1
297,0,1,0.028894,0.570569,0,0,0.99972,0.99972,1
298,0,3,0.097698,0.267243,0,0,0.998641,0.998641,1


In [233]:
A_correct_B_wrong

,y_label,y_A,y_prob_correct_A,y_prob_A,A,y_B,y_prob_correct_B,y_prob_B,B
12,5,5,0.315315,0.315315,1,3,0.053655,0.753414,0
36,4,4,0.326334,0.326334,1,7,0.003052,0.996187,0
76,9,9,0.333115,0.333115,1,0,0.167144,0.832351,0
101,5,5,0.259005,0.259005,1,7,0.104662,0.513199,0
171,9,9,0.430742,0.430742,1,3,0.010039,0.527993,0
178,5,5,0.220996,0.220996,1,3,0.000207,0.999753,0
184,3,3,0.23282,0.23282,1,0,0.165736,0.762415,0
247,9,9,0.368243,0.368243,1,8,0.000003,0.999997,0
286,1,1,0.616593,0.616593,1,8,0.368821,0.630906,0


In [234]:
A_wrong_B_wrong

,y_label,y_A,y_prob_correct_A,y_prob_A,A,y_B,y_prob_correct_B,y_prob_B,B
35,2,1,0.02934,0.325694,0,1,0.0,0.93951,0
47,9,1,0.189299,0.243881,0,8,0.038373,0.959992,0
53,3,9,0.105674,0.363836,0,6,0.000521,0.99941,0
57,7,3,0.043962,0.329053,0,5,0.0,0.99999,0
58,4,5,0.098013,0.228652,0,3,0.003957,0.98514,0
59,6,1,0.031643,0.327265,0,7,0.23121,0.679919,0
61,3,5,0.116148,0.219245,0,5,0.268519,0.731466,0
85,5,3,0.190139,0.225199,0,7,0.142349,0.801315,0
87,7,1,0.038673,0.414504,0,0,0.000032,0.995287,0
118,2,9,0.017582,0.376003,0,3,0.00156,0.993404,0
